<a href="https://colab.research.google.com/github/Mihir1703/Deep-Learning/blob/master/Fruit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                             title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                 5MB  2022-11-13 15:47:17          14260        408  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india    Credit Card Spending Habits in India           319KB  2022-12-14 07:30:37           1189         48  1.0              
michals22/coffee-dataset                                        Coffee dataset                                  24KB  2022-12-15 20:02:12           3347         76  1.0              
thedevastator/unlock-prof

In [2]:
!kaggle datasets download -d moltean/fruits

fruits.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!mkdir datasets
!unzip -q fruits.zip -d datasets

mkdir: cannot create directory ‘datasets’: File exists
replace datasets/fruits-360-original-size/fruits-360-original-size/Meta/apple_6/info.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import keras.utils as image
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
classes = os.listdir('datasets/fruits-360_dataset/fruits-360/Training')
classes[63],len(classes)

('Chestnut', 131)

In [7]:
train_batch=ImageDataGenerator().flow_from_directory(directory='datasets/fruits-360_dataset/fruits-360/Training',target_size=(229,229),classes=classes)

Found 67692 images belonging to 131 classes.


In [8]:
label_dict = dict((v,k) for k,v in train_batch.class_indices.items())
label_dict[45]

'Cherry Wax Black'

In [9]:
model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5",trainable=False),
    Dense(units=len(classes),activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, decay=0.001 / (5 * 0.5)) , metrics=['accuracy'])
model.build([None, 299, 299, 3])

In [10]:
if os.path.isfile('fruit.h5') is False:
    with tf.device("/device:GPU:0"):
        model.fit(train_batch,epochs=5)
    model.save('fruit.h5')

In [11]:
load_model = keras.models.load_model(
    'fruit.h5', custom_objects={'KerasLayer':hub.KerasLayer}, compile=True, options=None
)

In [15]:
import keras.utils as image

test_path = 'datasets/fruits-360_dataset/fruits-360/Test/'
orignal = []
predict = []

label_code = train_batch.class_indices
print(label_code,label_dict)

def process_img(file):
    img = image.load_img(file,target_size=(229,229))
    return np.array(img)


for i in os.listdir(test_path):
    batch = []
    for j in os.listdir(test_path+i):
        path = test_path+i+'/'+j
        img = process_img(path)
        batch.append(img)
        orignal.append(label_code[i])
    with tf.device("/device:GPU:0"):
        pred = np.argmax(model.predict(np.array(batch)),axis=1)
    predict = predict + pred.tolist()
    print("Batch ",i)

{'Pineapple': 0, 'Tomato not Ripened': 1, 'Tomato Maroon': 2, 'Mulberry': 3, 'Apple Golden 2': 4, 'Pear': 5, 'Onion Red': 6, 'Nectarine Flat': 7, 'Huckleberry': 8, 'Apple Golden 1': 9, 'Pear Abate': 10, 'Plum 2': 11, 'Lychee': 12, 'Pear Forelle': 13, 'Avocado': 14, 'Mandarine': 15, 'Pepper Orange': 16, 'Hazelnut': 17, 'Tomato 4': 18, 'Cherry 2': 19, 'Passion Fruit': 20, 'Apple Red Delicious': 21, 'Nut Forest': 22, 'Potato White': 23, 'Blueberry': 24, 'Raspberry': 25, 'Dates': 26, 'Tangelo': 27, 'Fig': 28, 'Watermelon': 29, 'Cantaloupe 1': 30, 'Kiwi': 31, 'Potato Red Washed': 32, 'Apple Red 2': 33, 'Onion White': 34, 'Eggplant': 35, 'Cherry Rainier': 36, 'Papaya': 37, 'Apple Braeburn': 38, 'Ginger Root': 39, 'Mango': 40, 'Nut Pecan': 41, 'Pomelo Sweetie': 42, 'Pepper Red': 43, 'Pear Kaiser': 44, 'Cherry Wax Black': 45, 'Walnut': 46, 'Pear Williams': 47, 'Rambutan': 48, 'Cherry Wax Yellow': 49, 'Lemon': 50, 'Cherry Wax Red': 51, 'Peach Flat': 52, 'Physalis with Husk': 53, 'Mangostan': 54

In [16]:
from sklearn.metrics import accuracy_score

print("Accuracy : ",accuracy_score(y_pred=predict,y_true=orignal))
predict,orignal

Accuracy :  0.008021861777150916


([117,
  28,
  112,
  117,
  117,
  117,
  69,
  112,
  117,
  117,
  112,
  117,
  69,
  28,
  112,
  28,
  117,
  69,
  112,
  117,
  117,
  117,
  117,
  117,
  69,
  117,
  69,
  112,
  69,
  117,
  69,
  117,
  112,
  117,
  117,
  28,
  69,
  69,
  112,
  112,
  117,
  117,
  112,
  117,
  117,
  69,
  112,
  69,
  69,
  69,
  69,
  112,
  69,
  117,
  90,
  69,
  69,
  69,
  69,
  69,
  117,
  69,
  112,
  90,
  117,
  117,
  69,
  69,
  117,
  69,
  117,
  112,
  117,
  117,
  117,
  69,
  28,
  117,
  117,
  117,
  117,
  117,
  69,
  69,
  69,
  112,
  112,
  117,
  69,
  117,
  69,
  69,
  69,
  117,
  117,
  117,
  117,
  117,
  69,
  117,
  69,
  112,
  117,
  69,
  117,
  117,
  28,
  112,
  69,
  117,
  69,
  117,
  117,
  117,
  112,
  117,
  117,
  69,
  28,
  69,
  69,
  117,
  112,
  117,
  112,
  117,
  69,
  117,
  117,
  69,
  117,
  90,
  117,
  69,
  117,
  69,
  117,
  112,
  28,
  112,
  69,
  112,
  112,
  69,
  117,
  69,
  117,
  117,
  117,
  117,
  117,
 